In [2]:
import cv2
import mediapipe as mp
import numpy as np

# T-shirt size chart (in cm)
tshirt_sizes = {
    "S": {"shoulder": 38, "chest": 90, "length": 65},
    "M": {"shoulder": 42, "chest": 98, "length": 70},
    "L": {"shoulder": 46, "chest": 106, "length": 74},
    "XL": {"shoulder": 50, "chest": 114, "length": 78}
}

selected_size = "S"  # You can change this manually or add dropdown later

# Load the T-shirt image with transparency
tshirt = cv2.imread("503.png")

def remove_white_background(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2BGRA)
    lower_white = np.array([200, 200, 200, 0], dtype=np.uint8)
    upper_white = np.array([255, 255, 255, 255], dtype=np.uint8)
    mask = cv2.inRange(image, lower_white, upper_white)
    image[mask == 255] = [0, 0, 0, 0]
    return image

if tshirt is None:
    raise FileNotFoundError("T-shirt image not found!")
tshirt = remove_white_background(tshirt)

def overlay_image(background, overlay, x, y, width, height):
    overlay = cv2.resize(overlay, (width, height))
    for i in range(height):
        for j in range(width):
            if y + i >= background.shape[0] or x + j >= background.shape[1]:
                continue
            alpha = overlay[i, j, 3] / 255.0
            if alpha > 0:
                background[y + i, x + j] = (1 - alpha) * background[y + i, x + j] + alpha * overlay[i, j, :3]
    return background

def calculate_distance(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))

def get_coords(landmarks, shape, index):
    h, w = shape[:2]
    lm = landmarks[index]
    return int(lm.x * w), int(lm.y * h)

def fit_color(value, expected, tolerance=5):
    diff = abs(value - expected)
    if diff <= tolerance:
        return (0, 255, 0)  # green
    elif diff <= tolerance * 2:
        return (0, 255, 255)  # yellow
    else:
        return (0, 0, 255)  # red

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
cap = cv2.VideoCapture(0)

# Calibration Step
print("Please hold a reference card (e.g. credit card, 8.5cm wide) in front of the camera.")
print("Press 'c' to capture it.")

pixels_per_cm = 15.0  # fallback default

while True:
    ret, frame = cap.read()
    if not ret:
        continue

    display = frame.copy()
    cv2.putText(display, "Hold card flat. Press 'c' to calibrate.", (30, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
    cv2.imshow("Calibration", display)

    key = cv2.waitKey(1)
    if key & 0xFF == ord('c'):
        break
    elif key & 0xFF in [ord('q'), 27]:
        exit()

r = cv2.selectROI("Select Card Width", frame, fromCenter=False, showCrosshair=True)
card_pixel_width = r[2]
known_card_width_cm = 8.5  # standard card width
pixels_per_cm = card_pixel_width / known_card_width_cm
print(f"Calibrated pixels/cm: {pixels_per_cm:.2f}")
cv2.destroyWindow("Select Card Width")

# Main Loop
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)

    if results.pose_landmarks:
        lm = results.pose_landmarks.landmark
        try:
            l_sh = get_coords(lm, frame.shape, mp_pose.PoseLandmark.LEFT_SHOULDER)
            r_sh = get_coords(lm, frame.shape, mp_pose.PoseLandmark.RIGHT_SHOULDER)
            l_el = get_coords(lm, frame.shape, mp_pose.PoseLandmark.LEFT_ELBOW)
            r_el = get_coords(lm, frame.shape, mp_pose.PoseLandmark.RIGHT_ELBOW)
            l_hip = get_coords(lm, frame.shape, mp_pose.PoseLandmark.LEFT_HIP)
            r_hip = get_coords(lm, frame.shape, mp_pose.PoseLandmark.RIGHT_HIP)

            shoulder_cm = calculate_distance(l_sh, r_sh) / pixels_per_cm
            chest_cm = calculate_distance(l_el, r_el) / pixels_per_cm
            torso_cm = calculate_distance(
                ((l_sh[0]+r_sh[0])//2, (l_sh[1]+r_sh[1])//2),
                ((l_hip[0]+r_hip[0])//2, (l_hip[1]+r_hip[1])//2)
            ) / pixels_per_cm

            # T-shirt overlay
            center_x = (l_sh[0] + r_sh[0]) // 2
            tshirt_width = int(abs(r_sh[0] - l_sh[0]) * 2)
            tshirt_height = int(tshirt_width * 1.4)
            x = max(0, center_x - tshirt_width // 2)
            y = max(0, min(l_sh[1], r_sh[1]) - int(tshirt_height * 0.2))
            frame = overlay_image(frame, tshirt, x, y, tshirt_width, tshirt_height)

            # Fit analysis
            target = tshirt_sizes[selected_size]
            cv2.putText(frame, f"Selected Size: {selected_size}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)

            for idx, (label, val, expected) in enumerate([
                ("Shoulder", shoulder_cm, target["shoulder"]),
                ("Chest", chest_cm, target["chest"]),
                ("Torso", torso_cm, target["length"])
            ]):
                color = fit_color(val, expected)
                bar_len = int(200 * min(val / expected, 1.5))
                y_pos = 60 + idx * 50
                cv2.putText(frame, f"{label}: {val:.1f}cm / {expected}cm", (10, y_pos), cv2.FONT_HERSHEY_SIMPLEX, 0.65, color, 2)
                cv2.rectangle(frame, (250, y_pos - 10), (250 + bar_len, y_pos + 10), color, -1)

        except:
            cv2.putText(frame, "Tracking Error", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2)

    cv2.imshow("E-Trial Room", frame)
    if cv2.waitKey(1) & 0xFF in [ord('q'), 27]:
        break

cap.release()
cv2.destroyAllWindows()


FileNotFoundError: T-shirt image not found!